# Задание

В этом задании вам предстоит дообучить трансформерную модель для NER-задачи в различных форматах:

1. Обучите NER-модель

- Загрузите набор данных [Collection5](https://github.com/natasha/corus?tab=readme-ov-file#load_ne5) - **1 балл**
- Разбейте набор данных на train/test части
- Дообучите модель [rubert-tiny2](https://huggingface.co/cointegrated/rubert-tiny2) на train-части корпуса для решения NER-задачи, сделайте замеры качества NER-метрик до и после дообучения - **2 балла**

2. Попробуйте улучшить качество модели следующими способами:
- Предварительно дообучите на train-части в MLM режиме, а потом дообучите на NER-задачу - **2 балла**
- Сгенерируйте синтетическую разметку* подходящего**, на ваш взгляд, новостного корпуса большой и умной моделью для русскоязычного NER***, а затем использовав ее для дообучения rubert-tiny2 вместе с основным набором данных - **2 балла**

3. Финально сравните результаты различных подходов - **1 балл**

*прогоните датасет через NER-модель, получите ее предсказания и используйте их в качестве резметки

**Можно использовать уже знакомый вам датасет lenta-ru, объем данных лучше взять от 10_000 текстов

***Например, можно взять модель модель DeepPavlov ner_collection3_bert. Инструкция по запуску есть в [документации](https://docs.deeppavlov.ai/en/master/features/models/NER.html)

**Общее**

- Принимаемые решения обоснованы (почему выбрана определенная архитектура/гиперпараметр/оптимизатор/преобразование и т.п.) - **1 балл**
- Обеспечена воспроизводимость решения: зафиксированы random_state, ноутбук воспроизводится от начала до конца без ошибок - **1 балл**

## Этап 0 - Подготовка

### Импортируем необходимые библиотеки и компоненты

In [1]:
import warnings
import re
import os
import random

from corus import load_ne5, load_lenta
from datasets import Dataset, ClassLabel, Sequence, concatenate_datasets
from natasha import Segmenter, NewsEmbedding, NewsNERTagger, Doc

import evaluate
import torch
import numpy as np
import pandas as pd


from transformers import (
    AutoTokenizer,
    AutoModelForTokenClassification,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    AutoModelForMaskedLM,
)


device = "cuda" if torch.cuda.is_available() else "cpu"


print(device)
warnings.filterwarnings("ignore")

cuda


### Фиксируем seed'ы

In [2]:
RANDOM_SEED = 42
os.environ["PYTHONHASHSEED"] = str(RANDOM_SEED)
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)

torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.cuda.manual_seed_all(RANDOM_SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

## Этап 1 - Загрузка данных

In [3]:
dataset = load_ne5("../data/hw_4/Collection5/")

In [4]:
for item in dataset:
    print(item)
    break

Ne5Markup(id='001', text='Россия рассчитывает на конструктивное воздействие США на Грузию\r\n\r\n04/08/2008 12:08\r\n\r\nМОСКВА, 4 авг - РИА Новости. Россия рассчитывает, что США воздействуют на Тбилиси в связи с обострением ситуации в зоне грузино-осетинского конфликта. Об этом статс-секретарь - заместитель министра иностранных дел России Григорий Карасин заявил в телефонном разговоре с заместителем госсекретаря США Дэниэлом Фридом.\r\n\r\n"С российской стороны выражена глубокая озабоченность в связи с новым витком напряженности вокруг Южной Осетии, противозаконными действиями грузинской стороны по наращиванию своих вооруженных сил в регионе, бесконтрольным строительством фортификационных сооружений", - говорится в сообщении.\r\n\r\n"Россия уже призвала Тбилиси к ответственной линии и рассчитывает также на конструктивное воздействие со стороны Вашингтона", - сообщил МИД России. ', spans=[Ne5Span(index='T1', type='GEOPOLIT', start=0, stop=6, text='Россия'), Ne5Span(index='T2', type='GE

### Минимальная предобработка данных

* Будем решать задачу как задачу классификации - набор токенов к метке.
* Для этого предобработаем наш текст до вида list токенов, list меток.
* Метки представим по схеме BIO.

In [5]:
def convert_markup_to_ner(item):
    text = item.text
    tokens = []
    offsets = []
    for match in re.finditer(r"\S+", text):
        tokens.append(match.group())
        offsets.append((match.start(), match.end()))

    labels = ["O"] * len(tokens)

    for span in item.spans:
        token_indices = [
            i for i, (tstart, tend) in enumerate(offsets) if not (tend <= span.start or tstart >= span.stop)
        ]
        if token_indices:
            labels[token_indices[0]] = "B-" + span.type
            for idx in token_indices[1:]:
                labels[idx] = "I-" + span.type

    return {"id": item.id, "tokens": tokens, "ner_tags": labels}

In [6]:
data_list = [convert_markup_to_ner(item) for item in dataset]

In [7]:
ner_dataset = Dataset.from_list(data_list)

In [8]:
unique_labels = set()
for example in ner_dataset:
    unique_labels.update(example["ner_tags"])

unique_labels = sorted(list(unique_labels))
label_to_id = {label: i for i, label in enumerate(unique_labels)}

In [9]:
label_to_id

{'B-GEOPOLIT': 0,
 'B-LOC': 1,
 'B-MEDIA': 2,
 'B-ORG': 3,
 'B-PER': 4,
 'I-GEOPOLIT': 5,
 'I-LOC': 6,
 'I-MEDIA': 7,
 'I-ORG': 8,
 'I-PER': 9,
 'O': 10}

In [10]:
def convert_labels(example):
    example["ner_tags"] = [label_to_id[label] for label in example["ner_tags"]]
    return example

In [11]:
ner_dataset = ner_dataset.map(convert_labels)

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

In [12]:
features = ner_dataset.features.copy()
features["ner_tags"] = Sequence(ClassLabel(names=unique_labels))
ner_dataset = ner_dataset.cast(features)

Casting the dataset:   0%|          | 0/999 [00:00<?, ? examples/s]

In [13]:
print(ner_dataset[0])

{'id': '002', 'tokens': ['Комиссар', 'СЕ', 'критикует', 'ограничительную', 'политику', 'в', 'отношении', 'беженцев', 'в', 'европейских', 'странах', '05/08/2008', '10:32', 'МОСКВА,', '5', 'августа', '/Новости-Грузия/.', 'Проводимая', 'в', 'европейских', 'странах', 'ограничительная', 'политика', 'в', 'отношении', 'беженцев', 'нарушает', 'ряд', 'международных', 'стандартов,', 'в', 'частности,', 'право', 'на', 'воссоединение', 'семей,', 'заявляет', 'Комиссар', 'Совета', 'Европы', 'по', 'правам', 'человека', 'Томас', 'Хаммарберг', '(Thomas', 'Hammarberg)', 'в', 'размещенном', 'на', 'его', 'сайте', 'еженедельном', 'комментарии.', '"Ограничительная', 'политика', 'в', 'отношении', 'беженцев', 'в', 'европейских', 'странах', 'уменьшает', 'возможности', 'воссоединения', 'разделенных', 'семей",', '-', 'полагает', 'он.', 'По', 'сообщению', 'РИА', 'Новости,', 'Хаммарберг', 'констатирует,', 'что', 'в', 'последнее', 'время', '"правительства', 'попытались', 'ограничить', 'приезд', 'близких', 'родственн

In [14]:
tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny2")
max_length = 128

### Токенизируем текст

Оставим оригинальные метки для первого токена в последовательности, остальные отметим -100.

In [15]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"],
        truncation=True,
        is_split_into_words=True,
        max_length=max_length,
        padding="max_length",
    )
    all_labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

In [16]:
tokenized_dataset = ner_dataset.map(tokenize_and_align_labels, batched=True)

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

In [17]:
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)

### Разделим данные на train-test

In [18]:
train_dataset = tokenized_dataset["train"]
test_dataset = tokenized_dataset["test"]

## Этап 2 - Обучение модели на задачу NER

In [19]:
num_labels = len(unique_labels)
model = AutoModelForTokenClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=num_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### Используем стандартные гиперпараметры для подобных моделей

Кажется, что модель переобучится за 10 эпох, но transformers сохранит лучший артефакт за нас :)

In [18]:
training_args = TrainingArguments(
    output_dir="../models/dbert",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

c:\Users\emperor\Documents\GitHub\AI-2024-2_Sem\DL NLP\.venv\lib\site-packages\transformers\training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


### Используем метрики классификации для sequence labeling

In [20]:
metric = evaluate.load("seqeval")

In [21]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    true_predictions = [
        [unique_labels[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [unique_labels[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

### Обучим модель

In [21]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

C:\Users\emperor\AppData\Local\Temp\ipykernel_13768\1377821198.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [22]:
print("Evaluation before fine-tuning:")
pre_training_results = trainer.evaluate()
print(pre_training_results)

Evaluation before fine-tuning:


{'eval_loss': 2.364283561706543, 'eval_model_preparation_time': 0.0021, 'eval_precision': 0.0151832055155318, 'eval_recall': 0.077500988533017, 'eval_f1': 0.02539189014121, 'eval_accuracy': 0.10832422481735028, 'eval_runtime': 0.4656, 'eval_samples_per_second': 429.544, 'eval_steps_per_second': 27.92}


In [23]:
trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,No log,0.883037,0.002100,0.000000,0.000000,0.000000,0.769948
2,No log,0.566144,0.002100,0.277066,0.247924,0.261686,0.835069
3,No log,0.434527,0.002100,0.381899,0.440490,0.409108,0.875511
4,No log,0.361122,0.002100,0.538299,0.605773,0.570047,0.908244
5,No log,0.314260,0.002100,0.603787,0.680902,0.640030,0.921935
6,No log,0.279122,0.002100,0.652441,0.713325,0.681526,0.930794
7,No log,0.256713,0.002100,0.675520,0.744168,0.708184,0.937353
8,No log,0.243194,0.002100,0.684814,0.756030,0.718662,0.940114
9,No log,0.236398,0.002100,0.688525,0.763938,0.724274,0.940862
10,0.470500,0.233604,0.002100,0.690519,0.763147,0.725019,0.941725


c:\Users\emperor\Documents\GitHub\AI-2024-2_Sem\DL NLP\.venv\lib\site-packages\seqeval\metrics\v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


TrainOutput(global_step=500, training_loss=0.4705190124511719, metrics={'train_runtime': 15.3424, 'train_samples_per_second': 520.779, 'train_steps_per_second': 32.589, 'total_flos': 14148015982080.0, 'train_loss': 0.4705190124511719, 'epoch': 10.0})

In [25]:
print("Evaluation after fine-tuning:")
post_training_results = trainer.evaluate()
print(post_training_results)

Evaluation after fine-tuning:
{'eval_loss': 0.2336038202047348, 'eval_model_preparation_time': 0.0021, 'eval_precision': 0.6905187835420393, 'eval_recall': 0.7631474891261368, 'eval_f1': 0.7250187828700226, 'eval_accuracy': 0.9417246735316114, 'eval_runtime': 0.5124, 'eval_samples_per_second': 390.31, 'eval_steps_per_second': 25.37, 'epoch': 10.0}


### Выводы по эксперименту

* Метрики после обучения значительно лучше, чем до, что очевидно.
* Судя по val_loss модель немного недообучилась.
* Получили очень высокий `accuracy` = 0.94, что понятно по метке 'O' - пустой, их большинство.
* Получили вполне неплохой `f1` = 0.725 на тесте, будем в дальнейшем сравнивать этот показатель.

## Этап 3 - Предобучим на MLM

### Подготовим данные

Склеим тексты, токенизируем их для нашей модели.

In [27]:
def join_tokens(example):
    example["text"] = " ".join(example["tokens"])
    return example


mlm_train_dataset = train_dataset.map(join_tokens)
mlm_test_dataset = test_dataset.map(join_tokens)

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

In [28]:
def tokenize_for_mlm(example):
    return tokenizer(example["text"], truncation=True, max_length=512, padding="max_length")


mlm_train_dataset = mlm_train_dataset.map(tokenize_for_mlm, batched=True)
mlm_train_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

mlm_test_dataset = mlm_test_dataset.map(tokenize_for_mlm, batched=True)
mlm_test_dataset.set_format(type="torch", columns=["input_ids", "attention_mask"])

Map:   0%|          | 0/999 [00:00<?, ? examples/s]

### Создадим модель и обучим её

Возьмём меньшее количество эпох, но чуть повысим LR.

In [31]:
mlm_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=0.15)
mlm_model = AutoModelForMaskedLM.from_pretrained("cointegrated/rubert-tiny2")

In [ ]:
mlm_training_args = TrainingArguments(
    output_dir="../models/mlm/",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    evaluation_strategy="epoch",
    learning_rate=5e-5,
    weight_decay=0.01,
)

mlm_trainer = Trainer(
    model=mlm_model,
    args=mlm_training_args,
    train_dataset=mlm_train_dataset,
    eval_dataset=mlm_test_dataset,
    data_collator=mlm_collator,
)

In [39]:
mlm_trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,2.945902
2,No log,2.821543
3,No log,2.891276


TrainOutput(global_step=171, training_loss=3.11281616924799, metrics={'train_runtime': 25.5505, 'train_samples_per_second': 105.555, 'train_steps_per_second': 6.693, 'total_flos': 20582750048256.0, 'train_loss': 3.11281616924799, 'epoch': 3.0})

### Промежуточный вывод

Успешно обучили модель MLM, проверим на NER.

### Обучаем NER модель с момента чекпоинта MLM

Оставим все оригинальные параметры. Возможно, это не оптимальный подход, но можно будет объективно сравнить.

In [40]:
model = AutoModelForTokenClassification.from_pretrained("../models/mlm/checkpoint-171/", num_labels=num_labels)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at ../models/mlm/checkpoint-171/ and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [43]:
ner_training_args = TrainingArguments(
    output_dir="../models/mlm/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

ner_trainer = Trainer(
    model=model,
    args=ner_training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [44]:
print("Evaluation before NER fine-tuning:")
print(ner_trainer.evaluate())

Evaluation before NER fine-tuning:


{'eval_loss': 2.375126600265503, 'eval_model_preparation_time': 0.0016, 'eval_precision': 0.027367955149011507, 'eval_recall': 0.14669829972321075, 'eval_f1': 0.04612993472179049, 'eval_accuracy': 0.12535235574987055, 'eval_runtime': 0.4322, 'eval_samples_per_second': 462.793, 'eval_steps_per_second': 30.082}


In [45]:
ner_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,No log,0.898690,0.001600,0.000000,0.000000,0.000000,0.769890
2,No log,0.573792,0.001600,0.295082,0.249110,0.270154,0.833688
3,No log,0.438180,0.001600,0.469182,0.496639,0.482520,0.878905
4,No log,0.360574,0.001600,0.581395,0.622776,0.601375,0.908186
5,No log,0.313215,0.001600,0.624777,0.691973,0.656660,0.924064
6,No log,0.276746,0.001600,0.662281,0.716489,0.688319,0.932233
7,No log,0.256119,0.001600,0.668948,0.734282,0.700094,0.936490
8,No log,0.243475,0.001600,0.673506,0.739818,0.705106,0.938100
9,No log,0.237623,0.001600,0.673797,0.747331,0.708661,0.938963
10,0.478200,0.234957,0.001600,0.677996,0.749308,0.711871,0.939539


TrainOutput(global_step=500, training_loss=0.478190673828125, metrics={'train_runtime': 15.0348, 'train_samples_per_second': 531.434, 'train_steps_per_second': 33.256, 'total_flos': 14148015982080.0, 'train_loss': 0.478190673828125, 'epoch': 10.0})

### Выводы

* Модель немного недоучилась, ровно как и в первом случае.
* Получили качество чуть хуже по `f1` = 0.712, но, вполне вероятно, при полном обучении, результаты были бы +- схожие.

## Этап 4 - Аугментация с помощью большой модели NER

* Используем 10_000 сэмплов lenta
* В качестве модели возьмём NewsNERTagger из natasha, он имеет хорошее качество по бенчмаркам при невероятной по сравнению с конкурентами производительности.

### Аналогично предобработаем данные

In [22]:
data_lenta = load_lenta("../data/raw/lenta-ru-news.csv.gz")

In [23]:
target_columns = ["title", "topic", "text"]
data_dict = {c: [] for c in target_columns}

for item in data_lenta:
    for column in target_columns:
        data_dict[column].append(eval(f"item.{column}"))

In [24]:
df = pd.DataFrame(data_dict)
df = df.sample(10_000, random_state=RANDOM_SEED)

In [25]:
dataset = Dataset.from_pandas(df)

### Используем намеченный natasha-сетап 

In [26]:
segmenter = Segmenter()
embedding = NewsEmbedding()
ner_tagger = NewsNERTagger(embedding)

In [27]:
def create_markup_natasha(example):
    text = example["text"]
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_ner(ner_tagger)

    tokens = []
    offsets = []
    for match in re.finditer(r"\S+", text):
        tokens.append(match.group())
        offsets.append((match.start(), match.end()))

    labels = ["O"] * len(tokens)

    for span in doc.spans:
        token_indices = [i for i, (s, e) in enumerate(offsets) if not (e <= span.start or s >= span.stop)]
        if token_indices:
            labels[token_indices[0]] = "B-" + span.type
            for idx in token_indices[1:]:
                labels[idx] = "I-" + span.type

    return {"tokens": tokens, "ner_tags": labels, "text": text}

In [28]:
markup_dataset = dataset.map(create_markup_natasha)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

### Приведём лейблы к виду нашего train датасета для объединения

In [29]:
label_list = [
    "B-GEOPOLIT",
    "B-LOC",
    "B-MEDIA",
    "B-ORG",
    "B-PER",
    "I-GEOPOLIT",
    "I-LOC",
    "I-MEDIA",
    "I-ORG",
    "I-PER",
    "O",
]
label_map = {label: i for i, label in enumerate(label_list)}


def convert_tags(example):
    example["ner_tags"] = [label_map[tag] for tag in example["ner_tags"]]
    return example


markup_dataset = markup_dataset.map(convert_tags)

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [30]:
features = markup_dataset.features.copy()
features["ner_tags"] = Sequence(feature=ClassLabel(names=label_list))
markup_dataset = markup_dataset.cast(features)

Casting the dataset:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [31]:
def tokenize_and_align_labels(batch):
    tokenized_inputs = tokenizer(
        batch["tokens"], truncation=True, padding="max_length", is_split_into_words=True, max_length=128
    )
    all_labels = []
    for i, labels in enumerate(batch["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:
                label_ids.append(labels[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        all_labels.append(label_ids)
    tokenized_inputs["labels"] = all_labels
    return tokenized_inputs

In [32]:
markup_dataset = markup_dataset.map(tokenize_and_align_labels, batched=True)
markup_dataset = markup_dataset.remove_columns(["title", "topic", "text"])

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

In [33]:
combined_train_dataset = concatenate_datasets([train_dataset, markup_dataset])

### Обучим модель

Параметры и метрики аналогичны предыдущим для чистоты эксперимента.

In [ ]:
unique_labels = set()
for example in combined_train_dataset:
    unique_labels.update(example["ner_tags"])
unique_labels = sorted(list(unique_labels))
unique_labels

In [34]:
model = AutoModelForTokenClassification.from_pretrained("cointegrated/rubert-tiny2", num_labels=len(unique_labels))

Some weights of BertForTokenClassification were not initialized from the model checkpoint at cointegrated/rubert-tiny2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [36]:
def compute_metrics_aug(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = []
    true_labels = []

    for pred, lab in zip(predictions, labels):
        pred_labels = []
        lab_labels = []
        for p_val, l_val in zip(pred, lab):
            if l_val != -100:
                pred_labels.append(label_list[p_val])
                lab_labels.append(label_list[l_val])
        true_predictions.append(pred_labels)
        true_labels.append(lab_labels)

    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [37]:
aug_training_args = TrainingArguments(
    output_dir="../models/augmented/",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.01,
)

aug_trainer = Trainer(
    model=model,
    args=aug_training_args,
    train_dataset=combined_train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics_aug,
)

In [38]:
print("Evaluation before Aug fine-tuning:")
print(aug_trainer.evaluate())

Evaluation before Aug fine-tuning:


{'eval_loss': 2.503415822982788, 'eval_model_preparation_time': 0.0, 'eval_precision': 0.019561815336463225, 'eval_recall': 0.09578544061302682, 'eval_f1': 0.032488628979857055, 'eval_accuracy': 0.06508063593732129, 'eval_runtime': 0.5807, 'eval_samples_per_second': 344.396, 'eval_steps_per_second': 22.386}


In [39]:
aug_trainer.train()

Epoch,Training Loss,Validation Loss,Model Preparation Time,Precision,Recall,F1,Accuracy
1,0.346200,0.263587,0.000000,0.653465,0.682759,0.667791,0.932060
2,0.101600,0.216964,0.000000,0.678821,0.706130,0.692207,0.937722
3,0.061300,0.182285,0.000000,0.694239,0.720307,0.707033,0.941153
4,0.051500,0.167431,0.000000,0.701548,0.729502,0.715252,0.944584
5,0.047500,0.158705,0.000000,0.721275,0.745594,0.733233,0.947615
6,0.038900,0.147120,0.000000,0.746026,0.773180,0.759360,0.951618
7,0.037200,0.145638,0.000000,0.752501,0.778161,0.765116,0.952877
8,0.033900,0.141626,0.000000,0.758442,0.783142,0.770594,0.954135
9,0.031800,0.139693,0.000000,0.762963,0.789272,0.775895,0.955107
10,0.030900,0.139085,0.000000,0.762593,0.788889,0.775518,0.955221


TrainOutput(global_step=6750, training_loss=0.06993983395894368, metrics={'train_runtime': 158.8719, 'train_samples_per_second': 679.73, 'train_steps_per_second': 42.487, 'total_flos': 191219555182080.0, 'train_loss': 0.06993983395894368, 'epoch': 10.0})

### Выводы

* Модель опять немного недоучилась
* Метрики качества значительно выше, особенно вырос `precision`
* `f1` = 0.776 - это явный лидер по метрикам

## Финальные выводы

* Смогли обучить весьма неплохую модель (если доучить, то был бы потенциал `f1` ~= 0.8)
* Аугментация данных с помощью большой модели показала себя очень эффективно, что подтверждается личным практическим опытом
* Предобучение MLM в данной задаче оказалось не очень полезно